In [ ]:
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets
import glob, cv2
import matplotlib.pyplot as plt
import os

In [ ]:
# pathology='pneumonia'
parent_directory='../'
parent_images_path=parent_directory+'/radiologist_audit/'
output_csv_path=parent_images_path+'radiologist_feedback_csvs/'

# os.makedirs(output_csv_path)

In [ ]:
pathology_names=['Others','normal',
 'pneumonia',
 'covid',
 'edema',
 'atelectasis',
 'tuberculosis',
 'fibrosis',
 'cardiomegaly',
 'pleural effusion',
 'surgical emphysema',
 'nodule',
 'pneumothorax',
 'scoliosis',
 'opaque hemithorax',
 'hernia',
 'pleural thickening',
'lung mass', 'ILD', 'consolidation', 'bronchietasis','fracture']

pathology_names=sorted(pathology_names)
pathology_names.insert(0, 'N/A')

question_option_dict = { '1. Does the image contain Pneumonia?': ['Yes', 'No', 'Uncertain'],
  '2. If yes, what is the severity of involvement?': ['N/A', 'Mild', 'Moderate', 'Severe'],
  '3. Which part of the Lung is involved?': ['N/A','LLL', 'LUL', 'RUL', 'RML', 'RLL'],
  '4. Does the mask coincide with the actual region of involvment?': ['N/A', 'No', 'Undercoverage', 'Perfect', 'Overcoverage', 'Exists in one, caught in both', 'Exists in both, caught in one'],
  '5. If the Image does/does not contain Pneumonia, is the model marking any other condition incorrectly?': pathology_names,
  '6. What class can the image be classified into given the prediction and reviewing radiologist as GT': ['TP', 'FN','TN','FP']
                       }

questions=list(question_option_dict.keys())
options=list(question_option_dict.values())

In [ ]:
question_option_dict = { 
'1. Is the Image abnormal?': ['Yes', 'No', 'Uncertain'],
'2. Does the mask coincide with the actual region of involvment?': ['N/A', 'No', 'Undercoverage', 'Perfect', 'Overcoverage', 'Exists in one, caught in both', 'Exists in both, caught in one'],
'3. Does the heatmap coincide with the actual region of involvment?': ['N/A', 'No', 'Undercoverage', 'Perfect', 'Overcoverage', 'Exists in one, caught in both', 'Exists in both, caught in one'],
'4. Which pathologies is the model correctly demarcating?': pathology_names,
'5. Which pathologies is the model missing out on?': pathology_names,
'6.  On the basis of the heatmap, is there a pathology where the model is less confident? ': pathology_names,
'7. What class can the image be classified into given the prediction and reviewing radiologist as GT': ['TP', 'FN','TN','FP']        
                       }

questions=list(question_option_dict.keys())
options=list(question_option_dict.values())

In [ ]:
def radiologist_interactive_feedback(images):
    df=pd.read_csv('../predicted_csvs/prediction_normal_platform.csv')
    df['filename']=df['image_name']
    df['filename']=df['filename'].apply(lambda x: x.split('/')[-1])
    df['filename']=df['filename'].apply(lambda x: 'normal_platform_'+x)
    
    
    for idx, i in enumerate(images):
        print (str(idx*100.0/len(images))+"% Done")
        image=cv2.imread(i)
        probability=str(df.loc[df['filename']==i.split('/')[-1]]['AI_probability'].values[0])
        fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(20,20))
        plt.subplot(121)
        plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
        plt.title(i.split('/')[-1]+"  :  "+probability)
        plt.show()

        print(questions[0])
        answer=widgets.Select(options=options[0], disabled=False)
        answer_0.append(answer)
        display(answer_0[idx])
        print('\n')

        print(questions[1])
        answer=widgets.Select(options=options[1], disabled=False)
        answer_1.append(answer)
        display(answer_1[idx])
        print('\n')

        print(questions[2])
        answer=widgets.SelectMultiple(options=options[2], disabled=False)
        answer_2.append(answer)
        display(answer_2[idx])
        print('\n')

        print(questions[3])
        answer=widgets.Select(options=options[3], disabled=False)
        answer_3.append(answer)
        display(answer_3[idx])
        print('\n')

        print(questions[4])
        answer=widgets.SelectMultiple(options=options[4], disabled=False)
        answer_4.append(answer)
        display(answer_4[idx])
        print('\n')

        print(questions[5])
        answer=widgets.Select(options=options[5], disabled=False)
        answer_5.append(answer)
        display(answer_5[idx])
        print('\n')

        print(questions[6])
        answer=widgets.Select(options=options[6], disabled=False)
        answer_6.append(answer)
        display(answer_6[idx])
        print('\n')        
        

def process_feedback(path, lists_of_answers):
    processed_answer_0=[i.value for i in lists_of_answers[0]]
    processed_answer_1=[i.value for i in lists_of_answers[1]]
    processed_answer_2=[i.value for i in lists_of_answers[2]]
    processed_answer_3=[i.value for i in lists_of_answers[3]]
    processed_answer_4=[i.value for i in lists_of_answers[4]]
    processed_answer_5=[i.value for i in lists_of_answers[5]]
    processed_answer_6=[i.value for i in lists_of_answers[6]]

    df=pd.DataFrame()
    df['Image_Path']=images
    df[questions[0]]=processed_answer_0
    df[questions[1]]=processed_answer_1
    df[questions[2]]=processed_answer_2
    df[questions[3]]=processed_answer_3
    df[questions[4]]=processed_answer_4
    df[questions[5]]=processed_answer_5
    df[questions[6]]=processed_answer_6
    
    df.to_csv(path+'_radiologist_feedback.csv')

# High Confidence Positive Images

In [ ]:
images=glob.glob(parent_images_path+'highly_confident_positive_images/lung_mass_*.png')

answer_0 = []
answer_1 = []
answer_2 = []
answer_3 = []
answer_4 = []
answer_5 = []
answer_6 = []

radiologist_interactive_feedback(images)

In [ ]:
lists_of_answers=[answer_0, answer_1, answer_2, answer_3, answer_4, answer_5, answer_6]

process_feedback(output_csv_path+'/highly_confident_positive_images', lists_of_answers)

# df.to_csv('./highly_confident_positive_lung_mass_radiologist_feedback.csv', index=None)

# Medium Confidence Positive Images

In [ ]:
images=glob.glob(parent_images_path+'medium_confident_positive_images/*.png')

answer_0 = []
answer_1 = []
answer_2 = []
answer_3 = []
answer_4 = []
answer_5 = []
answer_6 = []

radiologist_interactive_feedback(images)

In [ ]:
lists_of_answers=[answer_0, answer_1, answer_2, answer_3, answer_4, answer_5, answer_6]

process_feedback(output_csv_path+'/medium_confident_positive_images', lists_of_answers)

# Low Confidence Positive Images

In [ ]:
images=glob.glob(parent_images_path+'low_confident_positive_images/*.png')

answer_0 = []
answer_1 = []
answer_2 = []
answer_3 = []
answer_4 = []
answer_5 = []
answer_6 = []

radiologist_interactive_feedback(images)

In [ ]:
lists_of_answers=[answer_0, answer_1, answer_2, answer_3, answer_4, answer_5, answer_6]

process_feedback(output_csv_path+'/low_confident_positive_images', lists_of_answers)

# High Confidence Negative Images

In [ ]:
images=glob.glob(parent_images_path+'highly_confident_negative_images/normal_platform_*.png')

answer_0 = []
answer_1 = []
answer_2 = []
answer_3 = []
answer_4 = []
answer_5 = []
answer_6 = []

radiologist_interactive_feedback(images[0:50])

In [ ]:
lists_of_answers=[answer_0, answer_1, answer_2, answer_3, answer_4, answer_5, answer_6]

process_feedback(output_csv_path+'/highly_confident_negative_images_normal_kaggle_fit', lists_of_answers)

In [ ]:
len(answer_4)

# Medium Confidence Negative Images

In [ ]:
images=glob.glob(parent_images_path+'medium_confident_negative_images/normal_platform_*.png')

answer_0 = []
answer_1 = []
answer_2 = []
answer_3 = []
answer_4 = []
answer_5 = []
answer_6 = []

radiologist_interactive_feedback(images)

In [ ]:
lists_of_answers=[answer_0, answer_1, answer_2, answer_3, answer_4, answer_5, answer_6]

process_feedback(output_csv_path+'/medium_confident_negative_images_normal_kaggle_fit', lists_of_answers)

# Low Confidence Negative Images

In [ ]:
images=glob.glob(parent_images_path+'low_confident_negative_images/normal_platform_*.png')

answer_0 = []
answer_1 = []
answer_2 = []
answer_3 = []
answer_4 = []
answer_5 = []
answer_6 = []

radiologist_interactive_feedback(images)

In [ ]:
lists_of_answers=[answer_0, answer_1, answer_2, answer_3, answer_4, answer_5, answer_6]

process_feedback(output_csv_path+'/low_confident_negative_images_normal_kaggle_fit', lists_of_answers)

# Collate All CSVs

In [ ]:
all_csvs=glob.glob(output_csv_path+'/*.csv')
df=pd.DataFrame()

for i in all_csvs:
    temp_df=pd.read_csv(i)
    df=pd.concat([df, temp_df])
    
df.to_csv(output_csv_path+'combined_feedback.csv')